In [1]:
import numpy as np
import sys,os
ia898path = os.path.abspath('../..')
print(ia898path)
if ia898path not in sys.path:
    sys.path.append(ia898path)
import ia898.src as ia

/Users/robertoalotufo/mylocalprojects/mylocalprojects


In [2]:
num_labels = 10          # 10 labels, from 1 to 10   

## Parte 1: Carregamento e visualização dos dados


In [3]:
import scipy.io
file_name = '/Users/robertoalotufo/Downloads/ex4data1.mat'
dataread = scipy.io.loadmat(file_name)
X = dataread['X']
y = dataread['y']
print(X.shape,y.shape)
print(X.dtype,X.min(),X.max())

(5000, 400) (5000, 1)
float64 -0.131963230199 1.12768829916


In [4]:
N = X.shape[0]
i = np.random.permutation(N)[:10]
print(i)
nb = ia.nbshow(10)
for (x,label) in zip(X[i],y[i]):
    f = ia.normalize(x.reshape(20,20)).T
    nb.nbshow(f,'%d' % (label,))
nb.nbshow()

[ 566  533  933 1914 3629 1780  699   76 1924 4046]


""
1
""
1
""
1
""
3
""
7
""


##  Part 2: Carregamento parâmetros já treinados


In [5]:
# Load the weights into variables Theta1 and Theta2
# Setup the parameters you will use for this exercise
input_layer_size  = 400  # 20x20 Input Images of Digits
hidden_layer_size = 25   # 25 hidden units

file_name = '/Users/robertoalotufo/Downloads/ex4weights.mat'
datapar = scipy.io.loadmat(file_name)
print(datapar.keys())
Theta1 = datapar['Theta1']
Theta2 = datapar['Theta2']
print(Theta1.dtype,Theta1.shape)
print(Theta2.dtype,Theta2.shape)

dict_keys(['__header__', '__version__', '__globals__', 'Theta1', 'Theta2'])
float64 (25, 401)
float64 (10, 26)


## Part 3: Calcula Custo (Feedforward)
    #  To the neural network, you should first start by implementing the
    #  feedforward part of the neural network that returns the cost only. You
    #  should complete the code in nnCostFunction.m to return cost. After
    #  implementing the feedforward to compute the cost, you can verify that
    #  your implementation is correct by verifying that you get the same cost
    #  as us for the fixed debugging parameters.
    #
    #  We suggest implementing the feedforward cost *without* regularization
    #  first so that it will be easier for you to debug. Later, in part 4, you
    #  will get to implement the regularized cost.

In [12]:
def nnCostFunction(param_list, 
                   num_labels,
                   X, y, vlambda):
    '''
    NNCOSTFUNCTION Implements the neural network cost function for a two layer
    neural network which performs classification
    (J grad) = NNCOSTFUNCTON(nn_params, hidden_layer_size, num_labels, X, y, lambda) 
    computes the cost and gradient of the neural network. The
    parameters for the neural network are "unrolled" into the vector
    nn_params and need to be converted back into the weight matrices. 
 
    The returned parameter grad should be a "unrolled" vector of the
    partial derivatives of the neural network.

    Reshape nn_params back into the parameters Theta1 and Theta2, the weight matrices
    for our 2 layer neural network
    '''

    Theta1 = param_list[0]
    Theta2 = param_list[1]
    # Setup some useful variables
    m = X.shape[0]
         
    # You need to return the following variables correctly 
    J = 0;
    Theta1_grad = np.zeros_like(Theta1)
    Theta2_grad = np.zeros_like(Theta2)

    # ====================== YOUR CODE HERE ======================
    # Instructions: You should complete the code by working through the
    #               following parts.
    #
    # Part 1: Feedforward the neural network and return the cost in the
    #         variable J. After implementing Part 1, you can verify that your
    #         cost function computation is correct by verifying the cost
    #         computed in ex4.m
    #
    # Part 2: Implement the backpropagation algorithm to compute the gradients
    #         Theta1_grad and Theta2_grad. You should return the partial derivatives of
    #         the cost function with respect to Theta1 and Theta2 in Theta1_grad and
    #         Theta2_grad, respectively. After implementing Part 2, you can check
    #         that your implementation is correct by running checkNNGradients
    #
    #         Note: The vector y passed into the function is a vector of labels
    #               containing values from 1..K. You need to map this vector into a 
    #               binary vector of 1's and 0's to be used with the neural network
    #               cost function.
    #
    #         Hint: We recommend implementing backpropagation using a for-loop
    #               over the training examples if you are implementing it for the 
    #               first time.
    #
    # Part 3: Implement regularization with the cost function and gradients.
    #
    #         Hint: You can implement this around the code for
    #               backpropagation. That is, you can compute the gradients for
    #               the regularization separately and then add them to Theta1_grad
    #               and Theta2_grad from Part 2.
    #
    # Cria categórico - One-Hot
    yy = np.zeros((m, num_labels))
    for i in np.arange(m):
        yy[i,y[i]-1] = 1.0

    # Forward propagation
    a1 = np.hstack((np.ones((m, 1)), X))
    a2 = np.hstack((np.ones((m, 1)), sigmoid(a1.dot(Theta1.T))))
    a3 = sigmoid(a2.dot(Theta2.T))

    # Cost computation
    J = - (yy * np.log(a3) + (1 - yy) * np.log(1 - a3)).sum() / m

    # ... add regularization
    Theta1_sq = Theta1[:,2:] * Theta1[:,2:]
    Theta2_sq = Theta2[:,2:] * Theta2[:,2:]
    R = 0.5 * vlambda * (Theta1_sq).sum() + (Theta2_sq).sum() / m
    J = J + R

    # Back propagation
    d3 = a3 - yy;
    d2 = (d3 * Theta2) * a2 * (1 - a2)
    d2 = d2[:,2:]

    # Gradient computation
    Delta1 = d2.T * a1
    Delta2 = d3.T * a2
    Theta1_grad = Delta1 / m
    Theta2_grad = Delta2 / m
                           
    # ... add regularization
    Theta1_grad[:,2:] = Theta1_grad[:,2:] + vlambda * Theta1[:,2:] / m
    Theta2_grad[:,2:] = Theta2_grad[:,2:] + vlambda * Theta2[:,2:] / m

    grad = [Theta1_grad, Theta2_grad]
    return (J, grad)


In [13]:
# Weight regularization parameter (we set this to 0 here)
vlambda = 0

J = nnCostFunction([Theta1, Theta2],
                   num_labels, X, y, vlambda);

print('Cost at parameters (loaded from ex4weights): %f \
         \n(this value should be about 0.287629)' % (J,))


NameError: name 'sigmoid' is not defined